In [2]:
#CNN1
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Embedding
from keras.layers import Convolution1D,MaxPooling1D, Flatten
from keras.datasets import imdb
from keras import backend as K
from sklearn.model_selection import train_test_split
import pandas as pd
from keras.utils.np_utils import to_categorical

from sklearn.preprocessing import Normalizer
from keras.models import Sequential
from keras.layers import Convolution1D, Dense, Dropout, Flatten, MaxPooling1D
from keras.utils import np_utils
import numpy as np
import h5py
from keras import callbacks
from keras.layers import LSTM, GRU, SimpleRNN
from keras.callbacks import CSVLogger
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder


traindata=pd.read_csv('UNSW_NB15_training_set.csv',skiprows=1,skipfooter=35069,names=['id','dur','proto','service','state','spkts','dpkts','sbytes','dbytes','rate','sttl','dttl','sload','dload','sloss','dloss','sinpkt','dinpkt','sjit','djit','swin','stcpb','dtcpb','dwin','tcprtt','synack','ackdat','smean','dmean','trans_depth','response_body_len','ct_srv_src','ct_state_ttl','ct_dst_ltm','ct_src_dport_ltm','ct_dst_sport_ltm','ct_dst_src_ltm','is_ftp_login','ct_ftp_cmd','ct_flw_http_mthd','ct_src_ltm','ct_srv_dst','is_sm_ips_ports','attack_cat','label'])
testdata=pd.read_csv('UNSW_NB15_training_set.csv',skiprows=140273,names=['id','dur','proto','service','state','spkts','dpkts','sbytes','dbytes','rate','sttl','dttl','sload','dload','sloss','dloss','sinpkt','dinpkt','sjit','djit','swin','stcpb','dtcpb','dwin','tcprtt','synack','ackdat','smean','dmean','trans_depth','response_body_len','ct_srv_src','ct_state_ttl','ct_dst_ltm','ct_src_dport_ltm','ct_dst_sport_ltm','ct_dst_src_ltm','is_ftp_login','ct_ftp_cmd','ct_flw_http_mthd','ct_src_ltm','ct_srv_dst','is_sm_ips_ports','attack_cat','label'])


for column in traindata.columns:
    if traindata[column].dtype == type(object):
        le = LabelEncoder()
        traindata[column] = le.fit_transform(traindata[column])

for column in testdata.columns:
    if testdata[column].dtype == type(object):
        le = LabelEncoder()
        testdata[column] = le.fit_transform(testdata[column])

X = traindata.iloc[:,1:44]
Y = traindata.iloc[:,44]
C = testdata.iloc[:,44]
T = testdata.iloc[:,1:44]

scaler = Normalizer().fit(X)
trainX = scaler.transform(X)

scaler = Normalizer().fit(T)
testT = scaler.transform(T)

y_train = np.array(Y)
y_test = np.array(C)

X_train = np.reshape(trainX, (trainX.shape[0],trainX.shape[1],1))
X_test = np.reshape(testT, (testT.shape[0],testT.shape[1],1))




lstm_output_size = 128

cnn = Sequential()
cnn.add(Convolution1D(64, 3, border_mode="same",activation="relu",input_shape=(43, 1)))
cnn.add(MaxPooling1D(pool_length=(2)))
cnn.add(Flatten())
cnn.add(Dense(128, activation="relu"))
cnn.add(Dropout(0.5))
cnn.add(Dense(1, activation="sigmoid"))
print(cnn.summary())
# define optimizer and objective, compile cnn


cnn.compile(loss="binary_crossentropy", optimizer="adam",metrics=['accuracy'])

# train
checkpointer = callbacks.ModelCheckpoint(filepath="results/cnn1/checkpoint-{epoch:02d}.hdf5", verbose=1, save_best_only=True, monitor='val_acc',mode='max')
csv_logger = CSVLogger('results/cnn1/cnntrainanalysis1.csv',separator=',', append=False)
cnn.fit(X_train, y_train, nb_epoch=25,validation_data=(X_test, y_test),callbacks=[checkpointer,csv_logger])
cnn.save("results/cnn1/cnn_model.hdf5")


C:\Users\Vamsi\AppData\Local\conda\conda\envs\PythonCPU\lib\site-packages\ipykernel_launcher.py:31: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
C:\Users\Vamsi\AppData\Local\conda\conda\envs\PythonCPU\lib\site-packages\ipykernel_launcher.py:68: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, activation="relu", input_shape=(43, 1), padding="same")`
C:\Users\Vamsi\AppData\Local\conda\conda\envs\PythonCPU\lib\site-packages\ipykernel_launcher.py:69: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
C:\Users\Vamsi\AppData\Local\conda\conda\envs\PythonCPU\lib\site-packages\ipykernel_launcher.py:83: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, 43, 64)            256       
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 21, 64)            0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1344)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               172160    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 129       
Total params: 172,545
Trainable params: 172,545
Non-trainable params: 0
_________________________________________________________________
None

In [4]:
#CNN2
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Embedding
from keras.layers import Convolution1D,MaxPooling1D, Flatten
from keras.datasets import imdb
from keras import backend as K
from sklearn.model_selection import train_test_split
import pandas as pd
from keras.utils.np_utils import to_categorical

from sklearn.preprocessing import Normalizer
from keras.models import Sequential
from keras.layers import Convolution1D, Dense, Dropout, Flatten, MaxPooling1D
from keras.utils import np_utils
import numpy as np
import h5py
from keras import callbacks
from keras.layers import GRU, GRU, SimpleRNN
from keras.callbacks import CSVLogger
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder


traindata=pd.read_csv('UNSW_NB15_training_set.csv',skiprows=1,skipfooter=35069,names=['id','dur','proto','service','state','spkts','dpkts','sbytes','dbytes','rate','sttl','dttl','sload','dload','sloss','dloss','sinpkt','dinpkt','sjit','djit','swin','stcpb','dtcpb','dwin','tcprtt','synack','ackdat','smean','dmean','trans_depth','response_body_len','ct_srv_src','ct_state_ttl','ct_dst_ltm','ct_src_dport_ltm','ct_dst_sport_ltm','ct_dst_src_ltm','is_ftp_login','ct_ftp_cmd','ct_flw_http_mthd','ct_src_ltm','ct_srv_dst','is_sm_ips_ports','attack_cat','label'])
testdata=pd.read_csv('UNSW_NB15_training_set.csv',skiprows=140273,names=['id','dur','proto','service','state','spkts','dpkts','sbytes','dbytes','rate','sttl','dttl','sload','dload','sloss','dloss','sinpkt','dinpkt','sjit','djit','swin','stcpb','dtcpb','dwin','tcprtt','synack','ackdat','smean','dmean','trans_depth','response_body_len','ct_srv_src','ct_state_ttl','ct_dst_ltm','ct_src_dport_ltm','ct_dst_sport_ltm','ct_dst_src_ltm','is_ftp_login','ct_ftp_cmd','ct_flw_http_mthd','ct_src_ltm','ct_srv_dst','is_sm_ips_ports','attack_cat','label'])


for column in traindata.columns:
    if traindata[column].dtype == type(object):
        le = LabelEncoder()
        traindata[column] = le.fit_transform(traindata[column])

for column in testdata.columns:
    if testdata[column].dtype == type(object):
        le = LabelEncoder()
        testdata[column] = le.fit_transform(testdata[column])

X = traindata.iloc[:,1:44]
Y = traindata.iloc[:,44]
C = testdata.iloc[:,44]
T = testdata.iloc[:,1:44]

scaler = Normalizer().fit(X)
trainX = scaler.transform(X)

scaler = Normalizer().fit(T)
testT = scaler.transform(T)

y_train = np.array(Y)
y_test = np.array(C)

X_train = np.reshape(trainX, (trainX.shape[0],trainX.shape[1],1))
X_test = np.reshape(testT, (testT.shape[0],testT.shape[1],1))




gru_output_size = 70

cnn = Sequential()
cnn.add(Convolution1D(64, 3, border_mode="same",activation="relu",input_shape=(43, 1)))
cnn.add(Convolution1D(64, 3, border_mode="same", activation="relu"))
cnn.add(MaxPooling1D(pool_length=(2)))
cnn.add(GRU(gru_output_size))
cnn.add(Dropout(0.1))
cnn.add(Dense(1, activation="sigmoid"))

# define optimizer and objective, compile cnn

cnn.compile(loss="binary_crossentropy", optimizer="adam",metrics=['accuracy'])

# train
checkpointer = callbacks.ModelCheckpoint(filepath="results/cnn2/checkpoint-{epoch:02d}.hdf5", verbose=1, save_best_only=True, monitor='val_acc',mode='max')
csv_logger = CSVLogger('results/cnn2/cnntrainanalysis2.csv',separator=',', append=False)
cnn.fit(X_train, y_train, nb_epoch=25, validation_data=(X_test, y_test),callbacks=[checkpointer,csv_logger])
cnn.save("results/cnn2/cnn_model.hdf5")


C:\Users\Vamsi\AppData\Local\conda\conda\envs\PythonCPU\lib\site-packages\ipykernel_launcher.py:31: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
C:\Users\Vamsi\AppData\Local\conda\conda\envs\PythonCPU\lib\site-packages\ipykernel_launcher.py:68: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, activation="relu", input_shape=(43, 1), padding="same")`
C:\Users\Vamsi\AppData\Local\conda\conda\envs\PythonCPU\lib\site-packages\ipykernel_launcher.py:69: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, activation="relu", padding="same")`
C:\Users\Vamsi\AppData\Local\conda\conda\envs\PythonCPU\lib\site-packages\ipykernel_launcher.py:70: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
C:\Users\Vamsi\AppData\Local\conda\conda\envs\PythonCPU\lib\site-packages\ipykernel_launcher.py:83: UserW

Train on 140272 samples, validate on 35069 samples
Epoch 1/25
140272/140272 [==============================] - 66s 473us/step - loss: 0.4342 - acc: 0.7678 - val_loss: 0.1594 - val_acc: 0.9715

Epoch 00001: val_acc improved from -inf to 0.97148, saving model to results/cnn2/checkpoint-01.hdf5
Epoch 2/25
140272/140272 [==============================] - 59s 418us/step - loss: 0.3785 - acc: 0.8219 - val_loss: 0.1087 - val_acc: 0.9925cc: - ETA: 6s - loss: - ETA: 0s - loss: 0.3786 - acc: 

Epoch 00002: val_acc improved from 0.97148 to 0.99247, saving model to results/cnn2/checkpoint-02.hdf5
Epoch 3/25
140272/140272 [==============================] - 62s 442us/step - loss: 0.3439 - acc: 0.8428 - val_loss: 0.0990 - val_acc: 0.9925

Epoch 00003: val_acc improved from 0.99247 to 0.99250, saving model to results/cnn2/checkpoint-03.hdf5
Epoch 4/25
140272/140272 [==============================] - 62s 443us/step - loss: 0.3354 - acc: 0.8445 - val_loss: 0.1434 - val_acc: 0.9886

Epoch 00004: val_acc 

In [6]:
#CNN3

from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Embedding
from keras.layers import Convolution1D,MaxPooling1D, Flatten
from keras.datasets import imdb
from keras import backend as K
from sklearn.model_selection import train_test_split
import pandas as pd
from keras.utils.np_utils import to_categorical

from sklearn.preprocessing import Normalizer
from keras.models import Sequential
from keras.layers import Convolution1D, Dense, Dropout, Flatten, MaxPooling1D
from keras.utils import np_utils
import numpy as np
import h5py
from keras import callbacks
from keras.layers import LSTM, GRU, SimpleRNN
from keras.callbacks import CSVLogger
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder


traindata=pd.read_csv('UNSW_NB15_training_set.csv',skiprows=1,skipfooter=35069,names=['id','dur','proto','service','state','spkts','dpkts','sbytes','dbytes','rate','sttl','dttl','sload','dload','sloss','dloss','sinpkt','dinpkt','sjit','djit','swin','stcpb','dtcpb','dwin','tcprtt','synack','ackdat','smean','dmean','trans_depth','response_body_len','ct_srv_src','ct_state_ttl','ct_dst_ltm','ct_src_dport_ltm','ct_dst_sport_ltm','ct_dst_src_ltm','is_ftp_login','ct_ftp_cmd','ct_flw_http_mthd','ct_src_ltm','ct_srv_dst','is_sm_ips_ports','attack_cat','label'])
testdata=pd.read_csv('UNSW_NB15_training_set.csv',skiprows=140273,names=['id','dur','proto','service','state','spkts','dpkts','sbytes','dbytes','rate','sttl','dttl','sload','dload','sloss','dloss','sinpkt','dinpkt','sjit','djit','swin','stcpb','dtcpb','dwin','tcprtt','synack','ackdat','smean','dmean','trans_depth','response_body_len','ct_srv_src','ct_state_ttl','ct_dst_ltm','ct_src_dport_ltm','ct_dst_sport_ltm','ct_dst_src_ltm','is_ftp_login','ct_ftp_cmd','ct_flw_http_mthd','ct_src_ltm','ct_srv_dst','is_sm_ips_ports','attack_cat','label'])


for column in traindata.columns:
    if traindata[column].dtype == type(object):
        le = LabelEncoder()
        traindata[column] = le.fit_transform(traindata[column])

for column in testdata.columns:
    if testdata[column].dtype == type(object):
        le = LabelEncoder()
        testdata[column] = le.fit_transform(testdata[column])

X = traindata.iloc[:,1:44]
Y = traindata.iloc[:,44]
C = testdata.iloc[:,44]
T = testdata.iloc[:,1:44]

scaler = Normalizer().fit(X)
trainX = scaler.transform(X)

scaler = Normalizer().fit(T)
testT = scaler.transform(T)

y_train = np.array(Y)
y_test = np.array(C)

X_train = np.reshape(trainX, (trainX.shape[0],trainX.shape[1],1))
X_test = np.reshape(testT, (testT.shape[0],testT.shape[1],1))


lstm_output_size = 128

cnn = Sequential()
cnn.add(Convolution1D(64, 3, border_mode="same",activation="relu",input_shape=(43, 1)))
cnn.add(Convolution1D(64, 3, border_mode="same", activation="relu"))
cnn.add(MaxPooling1D(pool_length=(2)))
cnn.add(Convolution1D(128, 3, border_mode="same", activation="relu"))
cnn.add(Convolution1D(128, 3, border_mode="same", activation="relu"))
cnn.add(MaxPooling1D(pool_length=(2)))
cnn.add(Flatten())
cnn.add(Dense(128, activation="relu"))
cnn.add(Dropout(0.5))
cnn.add(Dense(1, activation="sigmoid"))

# define optimizer and objective, compile cnn

cnn.compile(loss="binary_crossentropy", optimizer="adam",metrics=['accuracy'])

# train
checkpointer = callbacks.ModelCheckpoint(filepath="results/cnn3/checkpoint-{epoch:02d}.hdf5", verbose=1, save_best_only=True, monitor='val_acc',mode='max')
csv_logger = CSVLogger('results/cnn3/cnntrainanalysis3.csv',separator=',', append=False)
cnn.fit(X_train, y_train, nb_epoch=25,validation_data=(X_test, y_test),callbacks=[checkpointer,csv_logger])
cnn.save("results/cnn3/cnn_model.hdf5")


C:\Users\Vamsi\AppData\Local\conda\conda\envs\PythonCPU\lib\site-packages\ipykernel_launcher.py:31: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
C:\Users\Vamsi\AppData\Local\conda\conda\envs\PythonCPU\lib\site-packages\ipykernel_launcher.py:66: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, activation="relu", input_shape=(43, 1), padding="same")`
C:\Users\Vamsi\AppData\Local\conda\conda\envs\PythonCPU\lib\site-packages\ipykernel_launcher.py:67: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, activation="relu", padding="same")`
C:\Users\Vamsi\AppData\Local\conda\conda\envs\PythonCPU\lib\site-packages\ipykernel_launcher.py:68: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
C:\Users\Vamsi\AppData\Local\conda\conda\envs\PythonCPU\lib\site-packages\ipykernel_launcher.py:69: UserW

Train on 140272 samples, validate on 35069 samples
Epoch 1/25
140272/140272 [==============================] - 234s 2ms/step - loss: 0.4253 - acc: 0.7809 - val_loss: 0.2021 - val_acc: 0.9623

Epoch 00001: val_acc improved from -inf to 0.96230, saving model to results/cnn3/checkpoint-01.hdf5
Epoch 2/25
140272/140272 [==============================] - 240s 2ms/step - loss: 0.3585 - acc: 0.8375 - val_loss: 0.1555 - val_acc: 0.9905

Epoch 00002: val_acc improved from 0.96230 to 0.99053, saving model to results/cnn3/checkpoint-02.hdf5
Epoch 3/25
140272/140272 [==============================] - 240s 2ms/step - loss: 0.3406 - acc: 0.8443 - val_loss: 0.2694 - val_acc: 0.9936

Epoch 00003: val_acc improved from 0.99053 to 0.99356, saving model to results/cnn3/checkpoint-03.hdf5
Epoch 4/25
140272/140272 [==============================] - 244s 2ms/step - loss: 0.3101 - acc: 0.8458 - val_loss: 0.0978 - val_acc: 0.9913oss: 0.3 - ETA: 1s - loss:

Epoch 00004: val_acc did not improve from 0.99356
Epo

In [ ]:
#CNN-gru1
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Embedding
from keras.layers import Convolution1D,MaxPooling1D, Flatten
from keras.datasets import imdb
from keras import backend as K
from sklearn.model_selection import train_test_split
import pandas as pd
from keras.utils.np_utils import to_categorical

from sklearn.preprocessing import Normalizer
from keras.models import Sequential
from keras.layers import Convolution1D, Dense, Dropout, Flatten, MaxPooling1D
from keras.utils import np_utils
import numpy as np
import h5py
from keras import callbacks
from keras.layers import GRU, GRU, SimpleRNN
from keras.callbacks import CSVLogger
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder


traindata=pd.read_csv('UNSW_NB15_training_set.csv',skiprows=1,skipfooter=35069,names=['id','dur','proto','service','state','spkts','dpkts','sbytes','dbytes','rate','sttl','dttl','sload','dload','sloss','dloss','sinpkt','dinpkt','sjit','djit','swin','stcpb','dtcpb','dwin','tcprtt','synack','ackdat','smean','dmean','trans_depth','response_body_len','ct_srv_src','ct_state_ttl','ct_dst_ltm','ct_src_dport_ltm','ct_dst_sport_ltm','ct_dst_src_ltm','is_ftp_login','ct_ftp_cmd','ct_flw_http_mthd','ct_src_ltm','ct_srv_dst','is_sm_ips_ports','attack_cat','label'])
testdata=pd.read_csv('UNSW_NB15_training_set.csv',skiprows=140273,names=['id','dur','proto','service','state','spkts','dpkts','sbytes','dbytes','rate','sttl','dttl','sload','dload','sloss','dloss','sinpkt','dinpkt','sjit','djit','swin','stcpb','dtcpb','dwin','tcprtt','synack','ackdat','smean','dmean','trans_depth','response_body_len','ct_srv_src','ct_state_ttl','ct_dst_ltm','ct_src_dport_ltm','ct_dst_sport_ltm','ct_dst_src_ltm','is_ftp_login','ct_ftp_cmd','ct_flw_http_mthd','ct_src_ltm','ct_srv_dst','is_sm_ips_ports','attack_cat','label'])


for column in traindata.columns:
    if traindata[column].dtype == type(object):
        le = LabelEncoder()
        traindata[column] = le.fit_transform(traindata[column])

for column in testdata.columns:
    if testdata[column].dtype == type(object):
        le = LabelEncoder()
        testdata[column] = le.fit_transform(testdata[column])

X = traindata.iloc[:,1:44]
Y = traindata.iloc[:,44]
C = testdata.iloc[:,44]
T = testdata.iloc[:,1:44]

scaler = Normalizer().fit(X)
trainX = scaler.transform(X)

scaler = Normalizer().fit(T)
testT = scaler.transform(T)

y_train = np.array(Y)
y_test = np.array(C)

X_train = np.reshape(trainX, (trainX.shape[0],trainX.shape[1],1))
X_test = np.reshape(testT, (testT.shape[0],testT.shape[1],1))

gru_output_size = 70

cnn = Sequential()
cnn.add(Convolution1D(64, 3, border_mode="same",activation="relu",input_shape=(43, 1)))
cnn.add(MaxPooling1D(pool_length=(2)))
cnn.add(GRU(gru_output_size))
cnn.add(Dropout(0.1))
cnn.add(Dense(1, activation="sigmoid"))

# define optimizer and objective, compile cnn

cnn.compile(loss="binary_crossentropy", optimizer="adam",metrics=['accuracy'])

# train
checkpointer = callbacks.ModelCheckpoint(filepath="results/cnn-gru1results/checkpoint-{epoch:02d}.hdf5", verbose=1, save_best_only=True, monitor='val_acc',mode='max')
csv_logger = CSVLogger('results/cnn-gru1results/cnntrainanalysis1.csv',separator=',', append=False)
cnn.fit(X_train, y_train, nb_epoch=25, validation_data=(X_test, y_test),callbacks=[checkpointer,csv_logger])
cnn.save("results/cnn-gru1results/cnn_model.hdf5")

'''

cnn.load_weights("results/cnn-gru1results/checkpoint-947.hdf5")


cnn.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
loss, accuracy = cnn.evaluate(X_test, y_test)
print("\nLoss: %.2f, Accuracy: %.2f%%" % (loss, accuracy*100))


y_pred = cnn.predict_classes(X_test)
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred , average="binary")
precision = precision_score(y_test, y_pred , average="binary")
f1 = f1_score(y_test, y_pred, average="binary")
np.savetxt('res/expected1.txt', y_test, fmt='%01d')
np.savetxt('res/predicted1.txt', y_pred, fmt='%01d')

print("confusion matrix")
print("----------------------------------------------")
print("accuracy")
print("%.6f" %accuracy)
print("racall")
print("%.6f" %recall)
print("precision")
print("%.6f" %precision)
print("f1score")
print("%.6f" %f1)
cm = metrics.confusion_matrix(y_test, y_pred)
print("==============================================")
'''


C:\Users\Vamsi\AppData\Local\conda\conda\envs\PythonCPU\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\Vamsi\AppData\Local\conda\conda\envs\PythonCPU\lib\site-packages\ipykernel_launcher.py:31: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.


C:\Users\Vamsi\AppData\Local\conda\conda\envs\PythonCPU\lib\site-packages\ipykernel_launcher.py:65: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, activation="relu", input_shape=(43, 1), padding="same")`
C:\Users\Vamsi\AppData\Local\conda\conda\envs\PythonCPU\lib\site-packages\ipykernel_launcher.py:66: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


C:\Users\Vamsi\AppData\Local\conda\conda\envs\PythonCPU\lib\site-packages\ipykernel_launcher.py:78: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 140272 samples, validate on 35069 samples
Epoch 1/25
140272/140272 [==============================] - 151s 1ms/step - loss: 0.4929 - acc: 0.7191 - val_loss: 0.1791 - val_acc: 0.8528

Epoch 00001: val_acc improved from -inf to 0.85283, saving model to results/cnn-gru1results/checkpoint-01.hdf5
Epoch 2/25
140272/140272 [==============================] - 163s 1ms/step - loss: 0.4397 - acc: 0.7645 - val_loss: 0.1258 - val_acc: 0.9885

Epoch 00002: val_acc improved from 0.85283 to 0.98845, saving model to results/cnn-gru1results/checkpoint-02.hdf5
Epoch 3/25
140272/140272 [==============================] - 202s 1ms/step - loss: 0.4247 - acc: 0.7841 - val_loss: 0.1303 - val_acc: 0.9689

Epoch 00003: val_acc did not improve from 0.98845
Epoch 4/25
140272/140272 [==============================] - 220s 2ms/step - loss: 0.4156 - acc: 0.7933 - val_loss: 0.1562 - val_acc: 0.9516

Epoch 00004: val_acc did not improve from 0.98845
Epoch 5/25
140272/140272 [==============================] - 

In [ ]:
#CNN-GRU2
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Embedding
from keras.layers import Convolution1D,MaxPooling1D, Flatten
from keras.datasets import imdb
from keras import backend as K
from sklearn.model_selection import train_test_split
import pandas as pd
from keras.utils.np_utils import to_categorical

from sklearn.preprocessing import Normalizer
from keras.models import Sequential
from keras.layers import Convolution1D, Dense, Dropout, Flatten, MaxPooling1D
from keras.utils import np_utils
import numpy as np
import h5py
from keras import callbacks
from keras.layers import GRU, GRU, SimpleRNN
from keras.callbacks import CSVLogger
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder


traindata=pd.read_csv('UNSW_NB15_training_set.csv',skiprows=1,skipfooter=35069,names=['id','dur','proto','service','state','spkts','dpkts','sbytes','dbytes','rate','sttl','dttl','sload','dload','sloss','dloss','sinpkt','dinpkt','sjit','djit','swin','stcpb','dtcpb','dwin','tcprtt','synack','ackdat','smean','dmean','trans_depth','response_body_len','ct_srv_src','ct_state_ttl','ct_dst_ltm','ct_src_dport_ltm','ct_dst_sport_ltm','ct_dst_src_ltm','is_ftp_login','ct_ftp_cmd','ct_flw_http_mthd','ct_src_ltm','ct_srv_dst','is_sm_ips_ports','attack_cat','label'])
testdata=pd.read_csv('UNSW_NB15_training_set.csv',skiprows=140273,names=['id','dur','proto','service','state','spkts','dpkts','sbytes','dbytes','rate','sttl','dttl','sload','dload','sloss','dloss','sinpkt','dinpkt','sjit','djit','swin','stcpb','dtcpb','dwin','tcprtt','synack','ackdat','smean','dmean','trans_depth','response_body_len','ct_srv_src','ct_state_ttl','ct_dst_ltm','ct_src_dport_ltm','ct_dst_sport_ltm','ct_dst_src_ltm','is_ftp_login','ct_ftp_cmd','ct_flw_http_mthd','ct_src_ltm','ct_srv_dst','is_sm_ips_ports','attack_cat','label'])


for column in traindata.columns:
    if traindata[column].dtype == type(object):
        le = LabelEncoder()
        traindata[column] = le.fit_transform(traindata[column])

for column in testdata.columns:
    if testdata[column].dtype == type(object):
        le = LabelEncoder()
        testdata[column] = le.fit_transform(testdata[column])

X = traindata.iloc[:,1:44]
Y = traindata.iloc[:,44]
C = testdata.iloc[:,44]
T = testdata.iloc[:,1:44]

scaler = Normalizer().fit(X)
trainX = scaler.transform(X)

scaler = Normalizer().fit(T)
testT = scaler.transform(T)

y_train = np.array(Y)
y_test = np.array(C)

X_train = np.reshape(trainX, (trainX.shape[0],trainX.shape[1],1))
X_test = np.reshape(testT, (testT.shape[0],testT.shape[1],1))

gru_output_size = 70

cnn = Sequential()
cnn.add(Convolution1D(64, 3, border_mode="same",activation="relu",input_shape=(43, 1)))
cnn.add(Convolution1D(64, 3, border_mode="same", activation="relu"))
cnn.add(MaxPooling1D(pool_length=(2)))
cnn.add(GRU(gru_output_size))
cnn.add(Dropout(0.1))
cnn.add(Dense(1, activation="sigmoid"))

# define optimizer and objective, compile cnn

cnn.compile(loss="binary_crossentropy", optimizer="adam",metrics=['accuracy'])

# train
checkpointer = callbacks.ModelCheckpoint(filepath="results/cnn-gru2results/checkpoint-{epoch:02d}.hdf5", verbose=1, save_best_only=True, monitor='val_acc',mode='max')
csv_logger = CSVLogger('results/cnn-gru2results/cnntrainanalysis2.csv',separator=',', append=False)
cnn.fit(X_train, y_train, nb_epoch=25,validation_data=(X_test, y_test),callbacks=[checkpointer,csv_logger])
cnn.save("results/cnn-gru2results/cnn_model.hdf5")


C:\Users\Vamsi\AppData\Local\conda\conda\envs\PythonCPU\lib\site-packages\ipykernel_launcher.py:31: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
C:\Users\Vamsi\AppData\Local\conda\conda\envs\PythonCPU\lib\site-packages\ipykernel_launcher.py:65: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, activation="relu", input_shape=(43, 1), padding="same")`
C:\Users\Vamsi\AppData\Local\conda\conda\envs\PythonCPU\lib\site-packages\ipykernel_launcher.py:66: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, activation="relu", padding="same")`
C:\Users\Vamsi\AppData\Local\conda\conda\envs\PythonCPU\lib\site-packages\ipykernel_launcher.py:67: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
C:\Users\Vamsi\AppData\Local\conda\conda\envs\PythonCPU\lib\site-packages\ipykernel_launcher.py:79: UserW

Train on 140272 samples, validate on 35069 samples
Epoch 1/25
140272/140272 [==============================] - 204s 1ms/step - loss: 0.4695 - acc: 0.7460 - val_loss: 0.1360 - val_acc: 0.9518

Epoch 00001: val_acc improved from -inf to 0.95178, saving model to results/cnn-gru2results/checkpoint-01.hdf5
Epoch 2/25
140272/140272 [==============================] - 284s 2ms/step - loss: 0.4243 - acc: 0.7825 - val_loss: 0.1605 - val_acc: 0.9541

Epoch 00002: val_acc improved from 0.95178 to 0.95412, saving model to results/cnn-gru2results/checkpoint-02.hdf5
Epoch 3/25
140272/140272 [==============================] - 432s 3ms/step - loss: 0.4170 - acc: 0.7894 - val_loss: 0.1461 - val_acc: 0.9484

Epoch 00003: val_acc did not improve from 0.95412
Epoch 4/25
140272/140272 [==============================] - 522s 4ms/step - loss: 0.4074 - acc: 0.7956 - val_loss: 0.1441 - val_acc: 0.9777

Epoch 00004: val_acc improved from 0.95412 to 0.97773, saving model to results/cnn-gru2results/checkpoint-04.h

In [2]:
#CNN-GRU3
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Embedding
from keras.layers import Convolution1D,MaxPooling1D, Flatten
from keras.datasets import imdb
from keras import backend as K
from sklearn.model_selection import train_test_split
import pandas as pd
from keras.utils.np_utils import to_categorical

from sklearn.preprocessing import Normalizer
from keras.models import Sequential
from keras.layers import Convolution1D, Dense, Dropout, Flatten, MaxPooling1D
from keras.utils import np_utils
import numpy as np
import h5py
from keras import callbacks
from keras.layers import GRU, GRU, SimpleRNN
from keras.callbacks import CSVLogger
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder


traindata=pd.read_csv('UNSW_NB15_training_set.csv',skiprows=1,skipfooter=35069,names=['id','dur','proto','service','state','spkts','dpkts','sbytes','dbytes','rate','sttl','dttl','sload','dload','sloss','dloss','sinpkt','dinpkt','sjit','djit','swin','stcpb','dtcpb','dwin','tcprtt','synack','ackdat','smean','dmean','trans_depth','response_body_len','ct_srv_src','ct_state_ttl','ct_dst_ltm','ct_src_dport_ltm','ct_dst_sport_ltm','ct_dst_src_ltm','is_ftp_login','ct_ftp_cmd','ct_flw_http_mthd','ct_src_ltm','ct_srv_dst','is_sm_ips_ports','attack_cat','label'])
testdata=pd.read_csv('UNSW_NB15_training_set.csv',skiprows=140273,names=['id','dur','proto','service','state','spkts','dpkts','sbytes','dbytes','rate','sttl','dttl','sload','dload','sloss','dloss','sinpkt','dinpkt','sjit','djit','swin','stcpb','dtcpb','dwin','tcprtt','synack','ackdat','smean','dmean','trans_depth','response_body_len','ct_srv_src','ct_state_ttl','ct_dst_ltm','ct_src_dport_ltm','ct_dst_sport_ltm','ct_dst_src_ltm','is_ftp_login','ct_ftp_cmd','ct_flw_http_mthd','ct_src_ltm','ct_srv_dst','is_sm_ips_ports','attack_cat','label'])


for column in traindata.columns:
    if traindata[column].dtype == type(object):
        le = LabelEncoder()
        traindata[column] = le.fit_transform(traindata[column])

for column in testdata.columns:
    if testdata[column].dtype == type(object):
        le = LabelEncoder()
        testdata[column] = le.fit_transform(testdata[column])

X = traindata.iloc[:,1:44]
Y = traindata.iloc[:,44]
C = testdata.iloc[:,44]
T = testdata.iloc[:,1:44]

scaler = Normalizer().fit(X)
trainX = scaler.transform(X)

scaler = Normalizer().fit(T)
testT = scaler.transform(T)

y_train = np.array(Y)
y_test = np.array(C)

X_train = np.reshape(trainX, (trainX.shape[0],trainX.shape[1],1))
X_test = np.reshape(testT, (testT.shape[0],testT.shape[1],1))




gru_output_size = 70

cnn = Sequential()
cnn.add(Convolution1D(64, 3, border_mode="same",activation="relu",input_shape=(43, 1)))
cnn.add(Convolution1D(64, 3, border_mode="same", activation="relu"))
cnn.add(MaxPooling1D(pool_length=(2)))
cnn.add(Convolution1D(128, 3, border_mode="same", activation="relu"))
cnn.add(Convolution1D(128, 3, border_mode="same", activation="relu"))
cnn.add(MaxPooling1D(pool_length=(2)))
cnn.add(GRU(gru_output_size))
cnn.add(Dropout(0.1))
cnn.add(Dense(1, activation="sigmoid"))

# define optimizer and objective, compile cnn

cnn.compile(loss="binary_crossentropy", optimizer="adam",metrics=['accuracy'])

# train
checkpointer = callbacks.ModelCheckpoint(filepath="results/cnn-gru3results/checkpoint-{epoch:02d}.hdf5", verbose=1, save_best_only=True, monitor='val_acc',mode='max')
csv_logger = CSVLogger('results/cnn-gru3results/cnntrainanalysis3.csv',separator=',', append=False)
cnn.fit(X_train, y_train, nb_epoch=25, validation_data=(X_test, y_test),callbacks=[checkpointer,csv_logger])
cnn.save("results/cnn-gru3results/cnn_model.hdf5")


C:\Users\Vamsi\AppData\Local\conda\conda\envs\PythonCPU\lib\site-packages\ipykernel_launcher.py:32: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
C:\Users\Vamsi\AppData\Local\conda\conda\envs\PythonCPU\lib\site-packages\ipykernel_launcher.py:69: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, activation="relu", input_shape=(43, 1), padding="same")`
C:\Users\Vamsi\AppData\Local\conda\conda\envs\PythonCPU\lib\site-packages\ipykernel_launcher.py:70: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, activation="relu", padding="same")`
C:\Users\Vamsi\AppData\Local\conda\conda\envs\PythonCPU\lib\site-packages\ipykernel_launcher.py:71: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
C:\Users\Vamsi\AppData\Local\conda\conda\envs\PythonCPU\lib\site-packages\ipykernel_launcher.py:72: UserW

Train on 140272 samples, validate on 35069 samples
Epoch 1/25
140272/140272 [==============================] - 166s 1ms/step - loss: 0.4380 - acc: 0.7738 - val_loss: 0.1461 - val_acc: 0.9303

Epoch 00001: val_acc improved from -inf to 0.93031, saving model to results/cnn-gru3results/checkpoint-01.hdf5
Epoch 2/25
140272/140272 [==============================] - 219s 2ms/step - loss: 0.3797 - acc: 0.8185 - val_loss: 0.1139 - val_acc: 0.9948

Epoch 00002: val_acc improved from 0.93031 to 0.99481, saving model to results/cnn-gru3results/checkpoint-02.hdf5
Epoch 3/25
140272/140272 [==============================] - 291s 2ms/step - loss: 0.3492 - acc: 0.8426 - val_loss: 0.1185 - val_acc: 0.9925

Epoch 00003: val_acc did not improve from 0.99481
Epoch 4/25
140272/140272 [==============================] - 312s 2ms/step - loss: 0.3434 - acc: 0.8469 - val_loss: 0.1348 - val_acc: 0.9896

Epoch 00004: val_acc did not improve from 0.99481
Epoch 5/25
140272/140272 [==============================] - 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Vamsi\AppData\Local\conda\conda\envs\PythonCPU\lib\site-packages\IPython\core\interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-8dac88d6ed18>", line 86, in <module>
    cnn.fit(X_train, y_train, nb_epoch=25, validation_data=(X_test, y_test),callbacks=[checkpointer,csv_logger])
  File "C:\Users\Vamsi\AppData\Local\conda\conda\envs\PythonCPU\lib\site-packages\keras\engine\training.py", line 1039, in fit
    validation_steps=validation_steps)
  File "C:\Users\Vamsi\AppData\Local\conda\conda\envs\PythonCPU\lib\site-packages\keras\engine\training_arrays.py", line 212, in fit_loop
    verbose=0)
  File "C:\Users\Vamsi\AppData\Local\conda\conda\envs\PythonCPU\lib\site-packages\keras\engine\training_arrays.py", line 392, in test_loop
    batch_outs = f(ins_batch)
  File "C:\Users\Vamsi\AppData\Local\conda\conda\envs\PythonCPU\lib\site-packages\keras\backend\tensorflow_b

KeyboardInterrupt: 

In [ ]:
#CNN-LSTM1
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Embedding
from keras.layers import Convolution1D,MaxPooling1D, Flatten
from keras.datasets import imdb
from keras import backend as K
from sklearn.model_selection import train_test_split
import pandas as pd
from keras.utils.np_utils import to_categorical

from sklearn.preprocessing import Normalizer
from keras.models import Sequential
from keras.layers import Convolution1D, Dense, Dropout, Flatten, MaxPooling1D
from keras.utils import np_utils
import numpy as np
import h5py
from keras import callbacks
from keras.layers import LSTM, GRU, SimpleRNN
from keras.callbacks import CSVLogger
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder


traindata=pd.read_csv('UNSW_NB15_training_set.csv',skiprows=1,skipfooter=35069,names=['id','dur','proto','service','state','spkts','dpkts','sbytes','dbytes','rate','sttl','dttl','sload','dload','sloss','dloss','sinpkt','dinpkt','sjit','djit','swin','stcpb','dtcpb','dwin','tcprtt','synack','ackdat','smean','dmean','trans_depth','response_body_len','ct_srv_src','ct_state_ttl','ct_dst_ltm','ct_src_dport_ltm','ct_dst_sport_ltm','ct_dst_src_ltm','is_ftp_login','ct_ftp_cmd','ct_flw_http_mthd','ct_src_ltm','ct_srv_dst','is_sm_ips_ports','attack_cat','label'])
testdata=pd.read_csv('UNSW_NB15_training_set.csv',skiprows=140273,names=['id','dur','proto','service','state','spkts','dpkts','sbytes','dbytes','rate','sttl','dttl','sload','dload','sloss','dloss','sinpkt','dinpkt','sjit','djit','swin','stcpb','dtcpb','dwin','tcprtt','synack','ackdat','smean','dmean','trans_depth','response_body_len','ct_srv_src','ct_state_ttl','ct_dst_ltm','ct_src_dport_ltm','ct_dst_sport_ltm','ct_dst_src_ltm','is_ftp_login','ct_ftp_cmd','ct_flw_http_mthd','ct_src_ltm','ct_srv_dst','is_sm_ips_ports','attack_cat','label'])


for column in traindata.columns:
    if traindata[column].dtype == type(object):
        le = LabelEncoder()
        traindata[column] = le.fit_transform(traindata[column])

for column in testdata.columns:
    if testdata[column].dtype == type(object):
        le = LabelEncoder()
        testdata[column] = le.fit_transform(testdata[column])

X = traindata.iloc[:,1:44]
Y = traindata.iloc[:,44]
C = testdata.iloc[:,44]
T = testdata.iloc[:,1:44]

scaler = Normalizer().fit(X)
trainX = scaler.transform(X)

scaler = Normalizer().fit(T)
testT = scaler.transform(T)

y_train = np.array(Y)
y_test = np.array(C)

X_train = np.reshape(trainX, (trainX.shape[0],trainX.shape[1],1))
X_test = np.reshape(testT, (testT.shape[0],testT.shape[1],1))

lstm_output_size = 70

cnn = Sequential()
cnn.add(Convolution1D(64, 3, border_mode="same",activation="relu",input_shape=(43, 1)))
cnn.add(MaxPooling1D(pool_length=(2)))
cnn.add(LSTM(lstm_output_size))
cnn.add(Dropout(0.1))
cnn.add(Dense(1, activation="sigmoid"))

# define optimizer and objective, compile cnn

cnn.compile(loss="binary_crossentropy", optimizer="adam",metrics=['accuracy'])

# train
checkpointer = callbacks.ModelCheckpoint(filepath="results/cnn-lstm1results/checkpoint-{epoch:02d}.hdf5", verbose=1, save_best_only=True, monitor='val_acc',mode='max')
csv_logger = CSVLogger('results/cnn-lstm1results/cnntrainanalysis1.csv',separator=',', append=False)
cnn.fit(X_train, y_train, nb_epoch=25,validation_data=(X_test, y_test),callbacks=[checkpointer,csv_logger])
cnn.save("results/cnn-lstm1results/cnn_model.hdf5")

'''

cnn.load_weights("results/cnn-lstm1results/checkpoint-947.hdf5")


cnn.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
loss, accuracy = cnn.evaluate(X_test, y_test)
print("\nLoss: %.2f, Accuracy: %.2f%%" % (loss, accuracy*100))


y_pred = cnn.predict_classes(X_test)
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred , average="binary")
precision = precision_score(y_test, y_pred , average="binary")
f1 = f1_score(y_test, y_pred, average="binary")
np.savetxt('res/expected1.txt', y_test, fmt='%01d')
np.savetxt('res/predicted1.txt', y_pred, fmt='%01d')

print("confusion matrix")
print("----------------------------------------------")
print("accuracy")
print("%.6f" %accuracy)
print("racall")
print("%.6f" %recall)
print("precision")
print("%.6f" %precision)
print("f1score")
print("%.6f" %f1)
cm = metrics.confusion_matrix(y_test, y_pred)
print("==============================================")
'''


C:\Users\Vamsi\AppData\Local\conda\conda\envs\PythonCPU\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\Vamsi\AppData\Local\conda\conda\envs\PythonCPU\lib\site-packages\ipykernel_launcher.py:32: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.


C:\Users\Vamsi\AppData\Local\conda\conda\envs\PythonCPU\lib\site-packages\ipykernel_launcher.py:66: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, activation="relu", input_shape=(43, 1), padding="same")`
C:\Users\Vamsi\AppData\Local\conda\conda\envs\PythonCPU\lib\site-packages\ipykernel_launcher.py:67: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


C:\Users\Vamsi\AppData\Local\conda\conda\envs\PythonCPU\lib\site-packages\ipykernel_launcher.py:79: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 140272 samples, validate on 35069 samples
Epoch 1/25
140272/140272 [==============================] - 197s 1ms/step - loss: 0.4848 - acc: 0.7162 - val_loss: 0.1946 - val_acc: 0.9691

Epoch 00001: val_acc improved from -inf to 0.96909, saving model to results/cnn-lstm1results/checkpoint-01.hdf5
Epoch 2/25
140272/140272 [==============================] - 217s 2ms/step - loss: 0.4487 - acc: 0.7534 - val_loss: 0.1370 - val_acc: 0.9982

Epoch 00002: val_acc improved from 0.96909 to 0.99818, saving model to results/cnn-lstm1results/checkpoint-02.hdf5
Epoch 3/25
140272/140272 [==============================] - 313s 2ms/step - loss: 0.4272 - acc: 0.7739 - val_loss: 0.1629 - val_acc: 0.9299

Epoch 00003: val_acc did not improve from 0.99818
Epoch 4/25
140272/140272 [==============================] - 413s 3ms/step - loss: 0.4175 - acc: 0.7833 - val_loss: 0.1482 - val_acc: 0.9867: 2s - 

Epoch 00004: val_acc did not improve from 0.99818
Epoch 5/25
140272/140272 [=========================

In [ ]:
#CNN-lstm2
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Embedding
from keras.layers import Convolution1D,MaxPooling1D, Flatten
from keras.datasets import imdb
from keras import backend as K
from sklearn.model_selection import train_test_split
import pandas as pd
from keras.utils.np_utils import to_categorical

from sklearn.preprocessing import Normalizer
from keras.models import Sequential
from keras.layers import Convolution1D, Dense, Dropout, Flatten, MaxPooling1D
from keras.utils import np_utils
import numpy as np
import h5py
from keras import callbacks
from keras.layers import LSTM, GRU, SimpleRNN
from keras.callbacks import CSVLogger
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder


traindata=pd.read_csv('UNSW_NB15_training_set.csv',skiprows=1,skipfooter=35069,names=['id','dur','proto','service','state','spkts','dpkts','sbytes','dbytes','rate','sttl','dttl','sload','dload','sloss','dloss','sinpkt','dinpkt','sjit','djit','swin','stcpb','dtcpb','dwin','tcprtt','synack','ackdat','smean','dmean','trans_depth','response_body_len','ct_srv_src','ct_state_ttl','ct_dst_ltm','ct_src_dport_ltm','ct_dst_sport_ltm','ct_dst_src_ltm','is_ftp_login','ct_ftp_cmd','ct_flw_http_mthd','ct_src_ltm','ct_srv_dst','is_sm_ips_ports','attack_cat','label'])
testdata=pd.read_csv('UNSW_NB15_training_set.csv',skiprows=140273,names=['id','dur','proto','service','state','spkts','dpkts','sbytes','dbytes','rate','sttl','dttl','sload','dload','sloss','dloss','sinpkt','dinpkt','sjit','djit','swin','stcpb','dtcpb','dwin','tcprtt','synack','ackdat','smean','dmean','trans_depth','response_body_len','ct_srv_src','ct_state_ttl','ct_dst_ltm','ct_src_dport_ltm','ct_dst_sport_ltm','ct_dst_src_ltm','is_ftp_login','ct_ftp_cmd','ct_flw_http_mthd','ct_src_ltm','ct_srv_dst','is_sm_ips_ports','attack_cat','label'])


for column in traindata.columns:
    if traindata[column].dtype == type(object):
        le = LabelEncoder()
        traindata[column] = le.fit_transform(traindata[column])

for column in testdata.columns:
    if testdata[column].dtype == type(object):
        le = LabelEncoder()
        testdata[column] = le.fit_transform(testdata[column])

X = traindata.iloc[:,1:44]
Y = traindata.iloc[:,44]
C = testdata.iloc[:,44]
T = testdata.iloc[:,1:44]

scaler = Normalizer().fit(X)
trainX = scaler.transform(X)

scaler = Normalizer().fit(T)
testT = scaler.transform(T)

y_train = np.array(Y)
y_test = np.array(C)

X_train = np.reshape(trainX, (trainX.shape[0],trainX.shape[1],1))
X_test = np.reshape(testT, (testT.shape[0],testT.shape[1],1))

lstm_output_size = 70

cnn = Sequential()
cnn.add(Convolution1D(64, 3, border_mode="same",activation="relu",input_shape=(43, 1)))
cnn.add(Convolution1D(64, 3, border_mode="same", activation="relu"))
cnn.add(MaxPooling1D(pool_length=(2)))
cnn.add(LSTM(lstm_output_size))
cnn.add(Dropout(0.1))
cnn.add(Dense(1, activation="sigmoid"))

# define optimizer and objective, compile cnn

cnn.compile(loss="binary_crossentropy", optimizer="adam",metrics=['accuracy'])

# train
checkpointer = callbacks.ModelCheckpoint(filepath="results/cnn-lstm2results/checkpoint-{epoch:02d}.hdf5", verbose=1, save_best_only=True, monitor='val_acc',mode='max')
csv_logger = CSVLogger('results/cnn-lstm2results/cnntrainanalysis2.csv',separator=',', append=False)
cnn.fit(X_train, y_train, nb_epoch=25,validation_data=(X_test, y_test),callbacks=[checkpointer,csv_logger])
cnn.save("results/cnn-lstm2results/cnn_model.hdf5")


C:\Users\Vamsi\AppData\Local\conda\conda\envs\PythonCPU\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\Vamsi\AppData\Local\conda\conda\envs\PythonCPU\lib\site-packages\ipykernel_launcher.py:32: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.


C:\Users\Vamsi\AppData\Local\conda\conda\envs\PythonCPU\lib\site-packages\ipykernel_launcher.py:66: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, activation="relu", input_shape=(43, 1), padding="same")`
C:\Users\Vamsi\AppData\Local\conda\conda\envs\PythonCPU\lib\site-packages\ipykernel_launcher.py:67: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, activation="relu", padding="same")`
C:\Users\Vamsi\AppData\Local\conda\conda\envs\PythonCPU\lib\site-packages\ipykernel_launcher.py:68: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


C:\Users\Vamsi\AppData\Local\conda\conda\envs\PythonCPU\lib\site-packages\ipykernel_launcher.py:80: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 140272 samples, validate on 35069 samples
Epoch 1/25
140272/140272 [==============================] - 192s 1ms/step - loss: 0.4805 - acc: 0.7151 - val_loss: 0.1798 - val_acc: 0.9871

Epoch 00001: val_acc improved from -inf to 0.98714, saving model to results/cnn-lstm2results/checkpoint-01.hdf5
Epoch 2/25
140272/140272 [==============================] - 248s 2ms/step - loss: 0.4255 - acc: 0.7783 - val_loss: 0.1709 - val_acc: 0.9358

Epoch 00002: val_acc did not improve from 0.98714
Epoch 3/25
140272/140272 [==============================] - 359s 3ms/step - loss: 0.4126 - acc: 0.7884 - val_loss: 0.1413 - val_acc: 0.9782

Epoch 00003: val_acc did not improve from 0.98714
Epoch 4/25
140272/140272 [==============================] - 421s 3ms/step - loss: 0.4130 - acc: 0.7914 - val_loss: 0.1476 - val_acc: 0.9291:  - ETA: 4:39 - loss: 0.4226  - ETA: 4:38 - - ET - ETA: 4:24 - loss: 0 - ETA: 4:23 - loss: 0.4219 - acc: - ETA: 4:22 - - ETA: 4:04 - loss: 0.4206 - acc - ETA: 4:03 - loss: 0.

In [ ]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility


from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Embedding
from keras.layers import Convolution1D,MaxPooling1D, Flatten
from keras.datasets import imdb
from keras import backend as K
from sklearn.model_selection import train_test_split
import pandas as pd
from keras.utils.np_utils import to_categorical

from sklearn.preprocessing import Normalizer
from keras.models import Sequential
from keras.layers import Convolution1D, Dense, Dropout, Flatten, MaxPooling1D
from keras.utils import np_utils
import numpy as np
import h5py
from keras import callbacks
from keras.layers import LSTM, GRU, SimpleRNN
from keras.callbacks import CSVLogger
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder


traindata=pd.read_csv('UNSW_NB15_training_set.csv',skiprows=1,skipfooter=35069,names=['id','dur','proto','service','state','spkts','dpkts','sbytes','dbytes','rate','sttl','dttl','sload','dload','sloss','dloss','sinpkt','dinpkt','sjit','djit','swin','stcpb','dtcpb','dwin','tcprtt','synack','ackdat','smean','dmean','trans_depth','response_body_len','ct_srv_src','ct_state_ttl','ct_dst_ltm','ct_src_dport_ltm','ct_dst_sport_ltm','ct_dst_src_ltm','is_ftp_login','ct_ftp_cmd','ct_flw_http_mthd','ct_src_ltm','ct_srv_dst','is_sm_ips_ports','attack_cat','label'])
testdata=pd.read_csv('UNSW_NB15_training_set.csv',skiprows=140273,names=['id','dur','proto','service','state','spkts','dpkts','sbytes','dbytes','rate','sttl','dttl','sload','dload','sloss','dloss','sinpkt','dinpkt','sjit','djit','swin','stcpb','dtcpb','dwin','tcprtt','synack','ackdat','smean','dmean','trans_depth','response_body_len','ct_srv_src','ct_state_ttl','ct_dst_ltm','ct_src_dport_ltm','ct_dst_sport_ltm','ct_dst_src_ltm','is_ftp_login','ct_ftp_cmd','ct_flw_http_mthd','ct_src_ltm','ct_srv_dst','is_sm_ips_ports','attack_cat','label'])


for column in traindata.columns:
    if traindata[column].dtype == type(object):
        le = LabelEncoder()
        traindata[column] = le.fit_transform(traindata[column])

for column in testdata.columns:
    if testdata[column].dtype == type(object):
        le = LabelEncoder()
        testdata[column] = le.fit_transform(testdata[column])

X = traindata.iloc[:,1:44]
Y = traindata.iloc[:,44]
C = testdata.iloc[:,44]
T = testdata.iloc[:,1:44]

scaler = Normalizer().fit(X)
trainX = scaler.transform(X)

scaler = Normalizer().fit(T)
testT = scaler.transform(T)

y_train = np.array(Y)
y_test = np.array(C)

X_train = np.reshape(trainX, (trainX.shape[0],trainX.shape[1],1))
X_test = np.reshape(testT, (testT.shape[0],testT.shape[1],1))




lstm_output_size = 70

cnn = Sequential()
cnn.add(Convolution1D(64, 3, border_mode="same",activation="relu",input_shape=(43, 1)))
cnn.add(Convolution1D(64, 3, border_mode="same", activation="relu"))
cnn.add(MaxPooling1D(pool_length=(2)))
cnn.add(Convolution1D(128, 3, border_mode="same", activation="relu"))
cnn.add(Convolution1D(128, 3, border_mode="same", activation="relu"))
cnn.add(MaxPooling1D(pool_length=(2)))
cnn.add(LSTM(lstm_output_size))
cnn.add(Dropout(0.1))
cnn.add(Dense(1, activation="sigmoid"))

# define optimizer and objective, compile cnn

cnn.compile(loss="binary_crossentropy", optimizer="adam",metrics=['accuracy'])

# train
checkpointer = callbacks.ModelCheckpoint(filepath="results/cnn-lstm3results/checkpoint-{epoch:02d}.hdf5", verbose=1, save_best_only=True, monitor='val_acc',mode='max')
csv_logger = CSVLogger('results/cnn-lstm3results/cnntrainanalysis3.csv',separator=',', append=False)
cnn.fit(X_train, y_train, nb_epoch=25, validation_data=(X_test, y_test),callbacks=[checkpointer,csv_logger])
cnn.save("results/cnn-lstm3results/cnn_model.hdf5")


C:\Users\Vamsi\AppData\Local\conda\conda\envs\PythonCPU\lib\site-packages\ipykernel_launcher.py:32: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
C:\Users\Vamsi\AppData\Local\conda\conda\envs\PythonCPU\lib\site-packages\ipykernel_launcher.py:69: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, activation="relu", input_shape=(43, 1), padding="same")`
C:\Users\Vamsi\AppData\Local\conda\conda\envs\PythonCPU\lib\site-packages\ipykernel_launcher.py:70: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, activation="relu", padding="same")`
C:\Users\Vamsi\AppData\Local\conda\conda\envs\PythonCPU\lib\site-packages\ipykernel_launcher.py:71: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
C:\Users\Vamsi\AppData\Local\conda\conda\envs\PythonCPU\lib\site-packages\ipykernel_launcher.py:72: UserW

Train on 140272 samples, validate on 35069 samples
Epoch 1/25
140272/140272 [==============================] - 238s 2ms/step - loss: 0.4378 - acc: 0.7751 - val_loss: 0.1651 - val_acc: 0.9488

Epoch 00001: val_acc improved from -inf to 0.94882, saving model to results/cnn-lstm3results/checkpoint-01.hdf5
Epoch 2/25
140272/140272 [==============================] - 323s 2ms/step - loss: 0.4090 - acc: 0.7957 - val_loss: 0.1363 - val_acc: 0.9603

Epoch 00002: val_acc improved from 0.94882 to 0.96031, saving model to results/cnn-lstm3results/checkpoint-02.hdf5
Epoch 3/25
140272/140272 [==============================] - 440s 3ms/step - loss: 0.3967 - acc: 0.8034 - val_loss: 0.1498 - val_acc: 0.9782

Epoch 00003: val_acc improved from 0.96031 to 0.97821, saving model to results/cnn-lstm3results/checkpoint-03.hdf5
Epoch 4/25
140272/140272 [==============================] - 587s 4ms/step - loss: 0.3625 - acc: 0.8335 - val_loss: 0.1499 - val_acc: 0.9748

Epoch 00004: val_acc did not improve from 0